In [5]:
# 🌡️🌧️ Análisis Climático INUMET - Temperatura y Precipitación (Corregido 100%)
# Preparado para Google Colab

import pandas as pd
import matplotlib.pyplot as plt

# -----------------------------
# Cargar datasets desde INUMET
# -----------------------------
url_temp = 'https://catalogodatos.gub.uy/dataset/accd0e24-76be-4101-904b-81bb7d41ee88/resource/f800fc53-556b-4d1c-8bd6-28b41f9cf146/download/inumet_temperatura_del_aire.csv'
url_prec = 'https://catalogodatos.gub.uy/dataset/fd896b11-4c04-4807-bae4-5373d65beea2/resource/ca987721-6052-4bb8-8596-2a5ad9630639/download/inumet_precipitacion_acumulada_horaria.csv'

# Cargar CSVs
temp = pd.read_csv(url_temp, encoding='ISO-8859-1', delimiter=';')
prec = pd.read_csv(url_prec, encoding='ISO-8859-1', delimiter=';')

# -----------------------------
# Limpiar nombres de columnas
# -----------------------------
temp.columns = temp.columns.str.strip().str.lower()  # quitar espacios y pasar a minúscula
prec.columns = prec.columns.str.strip().str.lower()

# Renombrar columnas clave según nombre real del CSV
temp.rename(columns={'temp_aire':'temperatura', 'estacion_id':'estacion'}, inplace=True)
prec.rename(columns={'precip_horario':'precipitacion', 'estacion_id':'estacion'}, inplace=True)

# Verificar columnas
print('Columnas temperatura:', temp.columns)
print('Columnas precipitación:', prec.columns)

# -----------------------------
# Convertir fechas a datetime
# -----------------------------
temp['fecha'] = pd.to_datetime(temp['fecha'], errors='coerce')
prec['fecha'] = pd.to_datetime(prec['fecha'], errors='coerce') # Changed 'fecha_hora' to 'fecha'


# Extraer solo fecha (sin hora) para unir
prec['fecha_simple'] = prec['fecha'].dt.date # Added a new column for date only
temp['fecha_simple'] = temp['fecha'].dt.date # Added a new column for date only
# Extraer solo hora para unir
prec['hour'] = prec['fecha'].dt.strftime('%H:%M')
# Added a new column for date only
temp['hour'] = temp['fecha'].dt.strftime('%H:%M')


# -----------------------------
# Revisar valores faltantes e imputar
# -----------------------------
temp['temperatura'] = temp['temperatura'].interpolate()
prec['precipitacion'] = prec['precipitacion'].fillna(0)

# -----------------------------
# Unir datasets por fecha y estación
# -----------------------------
df_merged = pd.merge(temp, prec, on=['fecha_simple','hour','estacion'], how='inner') # Merging on 'fecha_simple and hour'
df_merged= df_merged[['fecha_x','fecha_simple','hour','estacion','temperatura','precipitacion']]
df_merged=df_merged.rename(columns={'fecha_x':'datetime','hour':'hora'})
df_merged = df_merged.dropna()


print(df_merged.head(10))

# -----------------------------
# Guardar CSV combinado
# -----------------------------
df_merged.to_csv('/content/inumet_temp_prec.csv', index=False)
print("CSV combinado guardado en '/content/inumet_temp_prec.csv'")

Columnas temperatura: Index(['fecha', 'estacion', 'temperatura'], dtype='object')
Columnas precipitación: Index(['fecha', 'estacion', 'precipitacion'], dtype='object')
              datetime fecha_simple   hora               estacion  \
7  2020-01-01 00:00:00   2020-01-01  00:00  Aeropuerto Melilla G3   
8  2020-01-01 01:00:00   2020-01-01  01:00  Aeropuerto Melilla G3   
9  2020-01-01 02:00:00   2020-01-01  02:00  Aeropuerto Melilla G3   
10 2020-01-01 03:00:00   2020-01-01  03:00  Aeropuerto Melilla G3   
11 2020-01-01 04:00:00   2020-01-01  04:00  Aeropuerto Melilla G3   
12 2020-01-01 05:00:00   2020-01-01  05:00  Aeropuerto Melilla G3   
13 2020-01-01 06:00:00   2020-01-01  06:00  Aeropuerto Melilla G3   
14 2020-01-01 07:00:00   2020-01-01  07:00  Aeropuerto Melilla G3   
15 2020-01-01 08:00:00   2020-01-01  08:00  Aeropuerto Melilla G3   
16 2020-01-01 09:00:00   2020-01-01  09:00  Aeropuerto Melilla G3   

    temperatura  precipitacion  
7          21.3            0.2  
8     

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Crear variable objetivo: temperatura siguiente hora
df_merged['temp_siguiente'] = df_merged['temperatura'].shift(-1)
df_merged.dropna(inplace=True)

# Variables predictoras y objetivo
X = df_merged[['temperatura', 'precipitacion']]
y = df_merged['temp_siguiente']

# Separar entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Evaluar
print("R²:", modelo.score(X_test, y_test))

# Predicción de la próxima hora
pred = modelo.predict(X_test[:5])
print(pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Crear variable binaria: lluvia próxima hora
df_merged['llueve_proxima'] = (df_merged['precipitacion'].shift(-1) > 0).astype(int)
df_merged['llueve_ahora'] = (df_merged['precipitacion'] > 0).astype(int)
df_merged.dropna(inplace=True)

X = df_merged[['temperatura', 'precipitacion']]
y = df_merged['llueve_proxima']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

print("Precisión:", modelo.score(X_test, y_test))

In [ ]:
from prophet import Prophet

df_prophet = pd.DataFrame({
    'ds': df_merged['fecha_x'],
    'y': df_merged['temperatura']
})

modelo = Prophet()
modelo.fit(df_prophet)

futuro = modelo.make_future_dataframe(periods=24*7, freq='H')  # 7 días adelante
pred = modelo.predict(futuro)
modelo.plot(pred)

In [ ]:
url_humedad2020 = 'https://catalogodatos.gub.uy/dataset/ecd6cb6f-67c4-4f2f-8dd7-9155fa10b2f9/resource/2f5009da-5099-4f87-8ac7-2c34fd3195e3/download/inumet_humedad-relativa-2020.csv'

hum2020 = pd.read_csv(url_humedad2020,encoding='ISO-8859-1', delimiter=',')

#mostrar las primeras filas
hum2020.head(10)


In [ ]:
url_humedad2021 = 'https://catalogodatos.gub.uy/dataset/ecd6cb6f-67c4-4f2f-8dd7-9155fa10b2f9/resource/9d61886b-8232-42af-b91b-684065d412fa/download/inumet_humedad-relativa-2021.csv'

hum2021 = pd.read_csv(url_humedad2021,encoding='ISO-8859-1', delimiter=',')

#mostrar las primeras filas
hum2021.head(10)